# django 불러오기

In [1]:
import os, sys
# PWD = os.getenv('PWD')
PWD = os.path.abspath('.')

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [2]:
PWD

'c:\\Code\\skkukdt_minzy\\webserver\\django\\mydjango_board'

In [4]:
init_django('mydjango_board') #project name

RuntimeError: populate() isn't reentrant

# model import

In [ ]:
from board.models import Board, Comment

# Insert or Update

In [ ]:
board = Board(
    title="게시글1",
    content="내용1",
)

In [ ]:
board.save()

In [ ]:
board

-> 내용 확인

#### Manager 객체 - objects

In [2]:
board = Board.objects.create(
    title="게시글2",
    content="내용2"
)

NameError: ignored

In [ ]:
board

In [1]:
board = Board.objects.create(
    title="게시글3",
    content="내용3"
)

NameError: ignored

In [ ]:
board

### 여러 게시글을 한번에 insert하기

In [ ]:
data = []
for i in range(10):
    board = Board(title=f"새로운 게시글{i}", content=f"여러게시글 insert {i}")
    data.append(board)
    


In [ ]:
print(data)

In [ ]:
Board.objects.bulk_create(data)

In [ ]:
board = Board(title="게시물 추가", content="내용")
board

In [ ]:
board.save()

# 조회하기

## raw query로 조회하기

In [ ]:
result = Board.objects.raw('SELECT * FROM board_board')
result

In [ ]:
for i in result[:3]:
    print(i)

## orm으로 조회하기

### 전체 조회하기

In [ ]:
board_list = Board.objects.all()

In [ ]:
print(board_list.query)

### filter 걸어서 조회하기(where)

In [ ]:
board_list = Board.objects.filter(title="게시물 추가")
board_list

In [ ]:
board_list = Board.objects.order_by('-created_at') # order by desc
board_list

In [ ]:
board_list = Board.objects.order_by('-created_at')[2:4] # order by desc limit(2) offset(2)
board_list

###  filter not 표시

In [ ]:
board_list = Board.objects.exclude(title="게시물 추가")
board_list

In [ ]:
from django.db.models import Q

0001  
1001  


-> 
and: 0001  
or: 1001  
not 0001 => 1110  
xor: 1000  

####  비트연산
- ~: not
- &: and
- |: or
- ^: xor

In [ ]:
board_list = Board.objects.filter(~Q(title="게시물 추가"))
board_list

## 조회하기 고급

### 다양한 조건

#### or

In [ ]:
board_list = Board.objects.filter(Q(title="게시글1") | Q(title="게시글2"))
board_list

##### and 와 연산자

- gte: greater than equal
- lte: less than equal
- lt: less than
- gt: greater than

In [ ]:
board_list = Board.objects.filter(Q(title__endswith="글1") & Q(id__gte=3))
board_list

In [ ]:
board_list = Board.objects.filter(Q(title__startswith="게시") & Q(created_at__year=2023))
board_list

# 관계 추가하기

1번 댓글에 댓글을 3가지 이상 입력하세요

In [ ]:
comment = Comment(
    board_id = 1,
    content = "댓글 내용2"
)
comment.save()

In [ ]:
board = Board.objects.get(id=1)
comment = Comment.objects.create(
    board=board,
    content = "댓글 내용3"
)

In [ ]:
for i in range(10):
    Comment.objects.create(
        board = Board.objects.get(id=2),
        content = f"댓글{i}"
    )

## 관계 조회하기

In [ ]:
Comment.objects.all()

In [ ]:
board_list = Board.objects.filter(id__lte=3)
board_list

In [ ]:
board = board_list[0]
board

###  board가 가지고 있는 comment 모두 조회

In [ ]:
board.comment_set.all()

In [ ]:
for board in board_list:
    print(board)
    print(board.comment_set.all())
    print('-'*10)

In [ ]:
board_list = Board.objects.filter(id__lte=5).prefetch_related('comment_set')
board_list

In [ ]:
comment_list = board_list[0].comment_set.all()
comment_list

In [ ]:
comment = comment_list[0]
comment.board

# transaction

In [ ]:
from django.db import transaction

In [ ]:
@transaction.atomic
def sample():
    board = Board.objects.create(
        title='게시글',
        content = "atomic 게시글"
    )
    Comment.objects.create(
        board=board,
        content="atomic 댓글"
    )

In [ ]:
sample()

In [ ]:
with transaction.atomic():
    pass

In [ ]:
file = open("sample.txt", 'wt')
file.write("파일 쓰기")
file.close()

In [ ]:
with open("sample.txt", 'wt') as f:
    f.write("파일 쓰기")

### decorator - @ = 함수다

In [ ]:
def new_decorator(func):
    def wrap():
        print("decorator 시작")
        func()
        print("decorator 종료")
    return wrap

In [ ]:
@new_decorator
def my_func():
    print('my_func')
my_func()

decorator 시작
my_func
decorator 종료


In [ ]:
def my_func2():
    print('my_func')

In [ ]:
new_decorator(my_func2)()

decorator 시작
my_func
decorator 종료
